In [1]:
import Function as f
import numpy as np
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.layers import LSTM,Input,Dense
from tensorflow.keras.models import Sequential


In [2]:
def lstm(note1,note2,note3):
# define an activation function

    # timesteps and number of features 
    timesteps = 5
    n_features = 1

    # define a simple lstm for
    model = Sequential()
    model.add(Input(shape=(timesteps,n_features)))
    model.add(LSTM(note1,return_sequences=True))
    model.add(LSTM(note2,return_sequences=False))
    model.add(Dense(note3,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mae')
    
    return model

def ann(note1,note2,note3):
    
    n_features = 5
    
    model = Sequential()
    model.add(Input(shape=(n_features,)))
    model.add(Dense(note1,'relu'))
    model.add(Dense(note2,'relu'))
    model.add(Dense(note3,'relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mae')
    
    return model

def generator_price_lstm(x_train, y_train, x_val, y_val, x_test, y_test, date_test,y_scaler,
              model_parameter, epochs, batch_size, thred):
    # initialize
    acc = []

    for j in range(thred):
        
        y_true = np.array([])
        y_pred = np.array([])
        date = np.array([])
        model = lstm(note1 = model_parameter[0],
                     note2=model_parameter[1],
                     note3=model_parameter[2])
        for i in range(len(y_train)):
        
            
            model.fit(x_train[i], y_train[i], validation_data=(x_val[i], y_val[i]),
                      epochs=epochs, batch_size=batch_size,verbose=0)

            predition = model.predict(x_test[i])

            model.reset_states()
            y_true = np.append(y_true, y_test[i])
            y_pred = np.append(y_pred, predition)
            date = np.append(date, date_test[i])
            

        y_true_transform = y_true * y_scaler[1] + y_scaler[0]
        y_pred_transform = y_pred * y_scaler[1] + y_scaler[0]
        
        acc.append(mean_absolute_error(y_true_transform, y_pred_transform))
        
    acc_min = np.min(acc)
  
    return acc_min

def generator_price_ann(x_train, y_train, x_val, y_val, x_test, y_test, date_test,y_scaler,
              model_parameter, epochs, batch_size, thred):
    # initialize
    acc = []

    for j in range(thred):
        
        model = ann(note1 = model_parameter[0],
                     note2=model_parameter[1],
                     note3=model_parameter[2])
        
        y_true = np.array([])
        y_pred = np.array([])
        date = np.array([])

        for i in range(len(y_train)):
            model.fit(x_train[i], y_train[i], validation_data=(x_val[i], y_val[i]),
                      epochs=epochs, batch_size=batch_size,verbose=0)

            predition = model.predict(x_test[i])

            model.reset_states()
            y_true = np.append(y_true, y_test[i])
            y_pred = np.append(y_pred, predition)
            date = np.append(date, date_test[i])
            
        y_true_transform = y_true * y_scaler[1] + y_scaler[0]
        y_pred_transform = y_pred * y_scaler[1] + y_scaler[0]
        
        acc.append(mean_absolute_error(y_true_transform, y_pred_transform))
        # print(j,mean_absolute_error(y_true, y_pred))
        
    acc_min = np.min(acc)
  
    return acc_min


In [3]:
# LSTM
times = 8
names = ['AGG','DBC','VIX','VTI']

small = []
medium1 = []
medium2 = []
large = []
    
for i in names:
    
    
    # loading data 
    x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler = f.data_generator_lstm(name=i,timesteps=5,
                                                                train_len=100,val_len=10,test_len=5,forward_len=5)
    # model 8/4/2
    model = [8,4,2]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    small.append(acc)
    
    # model 32/16
    model = [32,16,8]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                               model,epochs=25,batch_size=50,thred=times)
    medium1.append(acc)
    
    # model 64/32
    model = [64,32,16]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    medium2.append(acc)
    
    # model 128/64
    model = [128,64,32]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    large.append(acc)

In [4]:
print(small)
print(medium1)
print(medium2)
print(large)

print(np.mean(small))
print(np.mean(medium1))
print(np.mean(medium2))
print(np.mean(large))
# 1.864068123257817
# 1.287479630961372
# 0.9085718958843285
# 0.8217149862388022

[0.903868473115469, 0.3568386077059097, 3.3165342149823367, 5.290401915884659]
[0.6789104219109491, 0.4237316432571268, 3.1323825257413525, 4.408298197318216]
[0.6765277688588053, 0.36818471484791576, 3.482339789891692, 3.716976469804802]
[0.7027150998478982, 0.38972657531883437, 3.6092197563472244, 3.9794226179768906]
2.4669108029220936
2.160830697056911
2.0610071858508037
2.170271012372712


In [5]:
# ANN
times = 8
names = ['AGG','DBC','VIX','VTI']

small = []
medium1 = []
medium2 = []
large = []
    
for i in names:
    
    # loading data 
    x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler= f.data_generator_ann(name=i,features=5,
                                                                train_len=100,val_len=10,test_len=5,forward_len=5)
    # model 8/4/2
    model = [8,4,2]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    small.append(acc)
    
    # model 32/16/8
    model = [32,16,8]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                                 model,epochs=25,batch_size=50,thred=times)
    medium1.append(acc)
    
    # model 64/32/16
    model = [64,32,16]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                              model,epochs=25,batch_size=50,thred=times)
    medium2.append(acc)
    
    # model 128/64/32
    model = [128,64,32]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    large.append(acc)

In [6]:
print(small)
print(medium1)
print(medium2)
print(large)

print(np.mean(small))
print(np.mean(medium1))
print(np.mean(medium2))
print(np.mean(large))

[0.7349189720420102, 0.5254900765800132, 3.3955527406653654, 5.358424683793011]
[0.5918354918409597, 0.30601411319091776, 2.7352683644564366, 3.6029049868381966]
[0.5931295820735342, 0.3367228420564297, 3.1516491956797226, 3.7627392304864022]
[0.6620748241326627, 0.3320437962140043, 3.130918772895896, 3.9598992714262446]
2.5035966182701
1.8090057390816277
1.9610602125740222
2.021234166167202
